In [25]:
import requests
from bs4 import BeautifulSoup
import time

from tqdm import tqdm

BASE_URL = "https://lipukule.org"

def get_post_links(page):
    """Собрать ссылки на все посты с главной (или из списка)."""
    resp = requests.get(BASE_URL + f"/page/{page}")
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    # На главной странице каждый заголовок ссылки — <a> внутри <h2> или <h3>? Нужно подстроить.
    links = []
    for a in soup.find_all("a"):
        href = a.get("href")
        if href and href.startswith("/") and len(href) > 1:
            # Простейшая фильтрация — относительные ссылки
            if not "/post/" in href:
                continue
            links.append(BASE_URL + href)
    # Можно дополнительно фильтровать по шаблону, чтобы оставить только посты
    links = set(links)
    return links

def parse_post(url):
    """Скачать страницу поста и извлечь текст."""
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    # Предположим, что тело статьи — внутри <div class="post-content"> либо просто <article>
    # Надо посмотреть HTML структуры сайта и подставить правильный селектор.
    content = ""
    # Пример:
    # main = soup.find("article")
    main = soup.find("div", class_="prose")
    if main is None:
        raise Exception("main is none")
        # попробовать другой селектор
        main = soup.find("div", class_="content")
    if main:
        # извлечь все текстовые узлы
        content = main.get_text(separator="\n", strip=True)
    else:
        # fallback — весь текст страницы
        content = soup.get_text(separator="\n", strip=True)
    return content


results = []
links = set()
for page in range(1, 8):
    links |= get_post_links(page)

print(f"Найдено {len(links)} ссылок.")
for link in tqdm(links):
    try:
        text = parse_post(link)
        results.append({"url": link, "text": text})
        time.sleep(0.5)
    except Exception as e:
        print("Ошибка при обработке", link, e)

Найдено 42 ссылок.


100%|██████████| 42/42 [00:43<00:00,  1.03s/it]


In [26]:
import polars as pl
from IPython.display import display


df = pl.DataFrame(results)
df = df.filter(~df["url"].str.contains_any(["/kule/", "/poki/", "/page/"]))

df

url,text
str,str
"""https://lipukule.org/post/2021…","""suli: nimi musi ni li monsuta …"
"""https://lipukule.org/post/2021…","""SITELEN NANPA WAN: kijetesanta…"
"""https://lipukule.org/post/2022…","""sina sona ala sona e ni: jan m…"
"""https://lipukule.org/post/2021…","""tonsi li seme? nimi ‘tonsi’ la…"
"""https://lipukule.org/post/2021…","""toki ilo forth li ilo wan ala.…"
…,…
"""https://lipukule.org/post/2021…","""tenpo pini la jan mute li jo a…"
"""https://lipukule.org/post/2022…","""mi ante e toki pi toki lili “A…"
"""https://lipukule.org/post/2024…","""nasin ni li pali e pan linja n…"


In [29]:
print(df["text"][0])

suli:
nimi musi ni li monsuta lili. ni li ike tawa sina la o lukin ala e ona.
tenpo pini mute la, kulupu li lon. kulupu ni li lili mute. jan lawa pi kulupu li jan Makalesi. ona li sona mute li lawa pona e kulupu. jan olin ona en ona li jo e jan lili wan. jan lili li jan Maku. ona li wile sona e ale li toki tawa jan ale. tenpo ale la, jan Maku li toki e ni: "ni li seme? tan seme la ni li lon? ma seme la ni li kama?"
mama pi jan Maku li sona ala e ijo la jan Maku li tawa tomo pi jan Olopa. tomo ona li lon poka pi ma kasi suli. jan Maku li pilin ike tan ma kasi suli. taso, jan Olopa li pona mute tawa ona. jan Olopa li jo e linja pi pimeja walo. ona li sona e ijo nasa mute. jan Maku li toki tawa jan Olopa la jan Olopa li pana e sona pi ijo nasa tawa ona. ni li pona tawa jan Maku. ona li wile sona e ijo nasa ale.
tenpo suno wan la, jan Maku li kute e nimi sin. ona li sona ala e nimi ni. nimi ni li pana e pilin nasa tawa jan Maku. taso, jan Maku li wile sona e kon nimi. jan Maku li tawa jan 

In [30]:
df.write_parquet("lipu-kule.pq")